# TP Partie 2 : Fine-tuning de LLM avec LoRA

## 📚 Contexte

Dans ce notebook, nous allons explorer le fine-tuning de modèles de langage avec la technique LoRA (Low-Rank Adaptation). Cette approche permet d'adapter des LLMs pour des tâches spécifiques.

## 🎯 Objectifs

1. **Comprendre** le fonctionnement de LoRA et ses avantages
2. **Préparer** le dataset synthétique généré précédemment
3. **Configurer** un entraînement LoRA avec Unsloth
4. **Entraîner** un modèle Qwen3-4B spécialisé en français
5. **Évaluer** les performances du modèle fine-tuné
6. **Publier** le modèle sur Hugging Face Hub

## 🔧 Etapes

Notre approche utilise :

1. **Unsloth** : Bibliothèque optimisée pour le fine-tuning LoRA
2. **Qwen3-4B** : Modèle de base pré-entraîné
3. **QLoRA** : Quantization + LoRA pour réduire la mémoire GPU
4. **Hugging Face Hub** : Publication et partage du modèle

Resources à garder sous la main durant le TP:

- [LoRA Hyperparameters Guide](https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/lora-hyperparameters-guide)
- [What Model Should I Use?](https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/what-model-should-i-use)
- [Qwen3 instruct fine tune guide](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-Instruct.ipynb)

In [1]:
# Import des bibliothèques
import json
import time
import os
from pathlib import Path
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from tqdm.auto import tqdm

# Imports Unsloth
try:
    from unsloth import FastLanguageModel
    from trl import SFTTrainer
    import trackio
    
    print("✅ Toutes les bibliothèques ont été importées avec succès!")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("Veuillez exécuter 'uv sync' pour installer les dépendances.")
    raise

# Vérification GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
# Configuration visuelle
sns.set_theme()
plt.rcParams['figure.figsize'] = (10, 6)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Toutes les bibliothèques ont été importées avec succès!
Device utilisé: cuda
GPU: NVIDIA L4
Mémoire GPU: 23.7 GB


## 3. Chargement et préparation du dataset

Nous chargeons le dataset synthétique généré dans la partie précédente.

In [2]:
# Configuration des chemins
data_dir = Path("../data/synthetic")

# Chargement du dataset
print("Chargement du dataset synthétique...")
if (data_dir / "synthetic_dataset_alpaca.json").exists():
    with open(data_dir / "synthetic_dataset_alpaca.json", 'r', encoding='utf-8') as f:
        alpaca_data = json.load(f)
    print(f"✅ Dataset chargé: {len(alpaca_data)} paires instruction-réponse")
else:
    print("❌ Dataset non trouvé. Veuillez exécuter le notebook 01 d'abord.")
    # Création d'un dataset d'exemple
    alpaca_data = [
        {
            "instruction": "Qu'est-ce que l'apprentissage automatique?",
            "input": "",
            "output": "L'apprentissage automatique est un sous-domaine de l'intelligence artificielle qui permet aux systèmes d'apprendre à partir de données sans être explicitement programmés."
        }
    ]

# Conversion en DataFrame pour analyse
df = pd.DataFrame(alpaca_data)
print(f"\n📊 Statistiques du dataset:")
print(f"   - Nombre total de paires: {len(df)}")
print(f"   - Longueur moyenne des instructions: {df['instruction'].str.len().mean():.0f} caractères")
print(f"   - Longueur moyenne des réponses: {df['output'].str.len().mean():.0f} caractères")

# Affichage des premières lignes
print("\n📝 Exemples de données:")
for i in range(min(3, len(alpaca_data))):
    print(f"\n--- Exemple {i+1} ---")
    print(f"Instruction: {alpaca_data[i]['instruction']}")
    print(f"Réponse: {alpaca_data[i]['output'][:100]}...")

Chargement du dataset synthétique...
✅ Dataset chargé: 36 paires instruction-réponse

📊 Statistiques du dataset:
   - Nombre total de paires: 36
   - Longueur moyenne des instructions: 102 caractères
   - Longueur moyenne des réponses: 919 caractères

📝 Exemples de données:

--- Exemple 1 ---
Instruction: En quelle année a eu lieu la consultation de l'élection étatique de Katanning ?
Réponse: La consultation de l'élection étatique de Katanning a eu lieu en **1935**....

--- Exemple 2 ---
Instruction: Quel est le nom du catégory de Wikimedia qui regroupe les icefalls de la région de Ross Dependency ?
Réponse: Le nom du catégory de Wikimedia qui regroupe les icefalls de la région de Ross Dependency est **Cate...

--- Exemple 3 ---
Instruction: Dans quel film Bourvil joue-t-il son dernier rôle, selon Jean Pierre Melville, et en quelle année a-t-il été tourné ?
Réponse: Selon Jean Pierre Melville, Bourvil joue son dernier rôle dans le film **"Le cercle rouge"**, qui a ...


## 4. Préparation des données pour l'entraînement

Nous formatons les données selon le format attendu par Unsloth pour le fine-tuning instructionnel.

In [3]:
# Template de formatage pour le modèle - aligné avec le format Unsloth/Qwen3
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Le EOS_TOKEN sera défini après le chargement du tokenizer
EOS_TOKEN = "<|im_end|>"  # Token de fin pour Qwen3

def format_prompts(examples):
    """
    Formate les exemples selon le template Alpaca pour le fine-tuning.
    """
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formatage du prompt
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

# Conversion en Dataset Hugging Face
dataset = Dataset.from_pandas(df)

# Application du formatage
print("Formatage des données pour l'entraînement...")
dataset = dataset.map(format_prompts, batched=True)

# Split train/validation
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

print(f"\n✅ Dataset préparé:")
print(f"   - Training set: {len(dataset_dict['train'])} exemples")
print(f"   - Validation set: {len(dataset_dict['validation'])} exemples")

# Affichage d'un exemple formaté
print("\n📝 Exemple formaté:")
print(dataset_dict['train'][0]['text'][:500] + "...")

Formatage des données pour l'entraînement...


Map:   0%|          | 0/36 [00:00<?, ? examples/s]


✅ Dataset préparé:
   - Training set: 32 exemples
   - Validation set: 4 exemples

📝 Exemple formaté:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Où se trouve le Barnes Ridge et dans quel état est-il situé ?

### Input:


### Response:
Le Barnes Ridge se trouve dans le **state de Montana**, aux États-Unis. Il est situé dans le **comté de Fergus County**, dans la partie centrale du pays, à environ 300 km au sud-ouest de Washington, D.C.<|im_end|>...


## 5. Configuration du modèle et de LoRA avec TrackIO

Nous configurons le modèle Qwen3-4B avec les paramètres LoRA optimisés selon les meilleures pratiques Unsloth, et initialisons TrackIO pour le suivi des expériences.

In [4]:
# Initialisation de TrackIO pour le suivi des expériences
experiment_name = f"qwen3-4b-lora-french-{int(time.time())}"
trackio.init(
    project="llm-finetuning-tp",
    name=experiment_name,
    config={
        "model": "Qwen/Qwen3-4B-Instruct-2507",
        "dataset": "synthetic_french",
        "framework": "unsloth",
        "optimization": "qlora"
    }
)

print(f"📊 TrackIO initialisé - Expérience: {experiment_name}")

# Configuration du modèle avec paramètres optimisés Unsloth
model_name = "Qwen/Qwen3-4B-Instruct-2507"
max_seq_length = 2048  # Longueur maximale de séquence
dtype = None  # Détection automatique du type
load_in_4bit = True  # Utilisation de la quantification 4-bit

print(f"Chargement du modèle {model_name}...")

# Chargement du modèle avec Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=None,  # Pas besoin de token pour les modèles publics
)

print("✅ Modèle chargé avec succès!")
print(f"   - Mémoire utilisée: {torch.cuda.memory_allocated() / 1e9:.1f} GB")
print(f"   - Mémoire réservée: {torch.cuda.memory_reserved() / 1e9:.1f} GB")

# Mettre à jour le EOS_TOKEN avec le tokenizer réel
global EOS_TOKEN
EOS_TOKEN = tokenizer.eos_token

# Log des informations système
trackio.log({
    "gpu_memory_allocated_gb": torch.cuda.memory_allocated() / 1e9,
    "gpu_memory_reserved_gb": torch.cuda.memory_reserved() / 1e9,
    "model_parameters_total": sum(p.numel() for p in model.parameters()),
    "model_size_gb": sum(p.numel() * p.element_size() for p in model.parameters()) / 1e9
})

* Trackio project initialized: llm-finetuning-tp
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "llm-finetuning-tp"
* or by running in Python: trackio.show(project="llm-finetuning-tp")
📊 TrackIO initialisé - Expérience: qwen3-4b-lora-french-1756829650
Chargement du modèle Qwen/Qwen3-4B-Instruct-2507...
==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Modèle chargé avec succès!
   - Mémoire utilisée: 3.6 GB
   - Mémoire réservée: 4.1 GB


In [5]:
# Configuration des paramètres LoRA optimisés selon les best practuces Unsloth
# Basé sur: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-Instruct.ipynb
lora_config = {
    "r": 16,  # Rank des matrices de mise à jour
    "target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # Modules à adapter - optimisé pour Qwen3
    "lora_alpha": 16,  # Facteur d'échelle (aligné avec le rank)
    "lora_dropout": 0,  # Pas de dropout pour une meilleure stabilité
    "bias": "none",  # Pas d'adaptation des biais
    "use_gradient_checkpointing": "unsloth",  # Optimisé Unsloth
    "random_state": 3407,
    "use_rslora": False,  # Ne pas utiliser scaled LoRA
    "loftq_config": None,  # Configuration LoftQ (pour quantification avancée)
}

print("Configuration LoRA:")
for key, value in lora_config.items():
    print(f"   - {key}: {value}")

# Application de la configuration LoRA
model = FastLanguageModel.get_peft_model(
    model,
    **lora_config
)

print("\n✅ Configuration LoRA appliquée!")

# Log des paramètres LoRA
trackio.log({
    "lora_rank": lora_config["r"],
    "lora_alpha": lora_config["lora_alpha"],
    "lora_dropout": lora_config["lora_dropout"],
    "target_modules": lora_config["target_modules"],
    "gradient_checkpointing": lora_config["use_gradient_checkpointing"]
})

# Affichage des paramètres entraînables
trainable_stats = model.print_trainable_parameters()
print(f"   - Paramètres entraînables: {trainable_stats}")

Configuration LoRA:
   - r: 16
   - target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
   - lora_alpha: 16
   - lora_dropout: 0
   - bias: none
   - use_gradient_checkpointing: unsloth
   - random_state: 3407
   - use_rslora: False
   - loftq_config: None


Unsloth 2025.8.10 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.



✅ Configuration LoRA appliquée!
trainable params: 33,030,144 || all params: 4,055,498,240 || trainable%: 0.8145
   - Paramètres entraînables: None


## 6. Configuration de l'entraînement

Nous configurons les hyperparamètres d'entraînement et le trainer.

In [6]:
# Configuration des arguments d'entraînement optimisés avec TrackIO
training_args = TrainingArguments(
    output_dir="./lora_finetuned_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=3,  # Nombre d'époques
    learning_rate=2e-4,  # Taux d'apprentissage optimal pour LoRA
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="trackio",  # Utilisation de TrackIO pour le tracking
    # Paramètres supplémentaires pour le monitoring
    logging_first_step=True,
    logging_dir="./logs",
    ddp_find_unused_parameters=False,
)

print("Arguments d'entraînement:")
print(f"   - Batch size: {training_args.per_device_train_batch_size}")
print(f"   - Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   - Learning rate: {training_args.learning_rate}")
print(f"   - Epochs: {training_args.num_train_epochs}")
print(f"   - Warmup steps: {training_args.warmup_steps}")
print(f"   - Report to: {training_args.report_to}")

# Log des hyperparamètres
trackio.log({
    "per_device_train_batch_size": training_args.per_device_train_batch_size,
    "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
    "learning_rate": training_args.learning_rate,
    "num_train_epochs": training_args.num_train_epochs,
    "warmup_steps": training_args.warmup_steps,
    "weight_decay": training_args.weight_decay,
    "optimizer": training_args.optim,
    "lr_scheduler_type": training_args.lr_scheduler_type,
    "effective_batch_size": training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
})

Arguments d'entraînement:
   - Batch size: 2
   - Gradient accumulation: 4
   - Learning rate: 0.0002
   - Epochs: 3
   - Warmup steps: 5
   - Report to: ['trackio']


In [7]:
# Configuration du trainer SFT de TRL
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Ne pas packer les séquences pour Qwen3
    args=training_args,
)

print("✅ SFTTrainer configuré!")
print(f"   - Échantillons d'entraînement: {len(trainer.train_dataset)}")
print(f"   - Échantillons de validation: {len(trainer.eval_dataset)}")
print(f"   - Sequence length: {max_seq_length}")
print(f"   - Packing: {False}")

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/32 [00:00<?, ? examples/s]

num_proc must be <= 4. Reducing num_proc to 4 for dataset of size 4.


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

✅ SFTTrainer configuré!
   - Échantillons d'entraînement: 32
   - Échantillons de validation: 4
   - Sequence length: 2048
   - Packing: False


## 7. Entraînement du modèle

**⚠️ Important** : Cette étape peut prendre du temps (15-30 minutes) selon votre GPU et la taille du dataset.

In [8]:
# Démarrage de l'entraînement avec monitoring TrackIO
print("🚀 Démarrage de l'entraînement LoRA...")
print(f"   - Dataset: {len(dataset_dict['train'])} exemples")
print(f"   - Paramètres LoRA: rank={lora_config['r']}, alpha={lora_config['lora_alpha']}")
print(f"   - Learning rate: {training_args.learning_rate}")
print(f"   - Epochs: {training_args.num_train_epochs}")
print(f"   - TrackIO: Activé\\n")

# Suivi de la mémoire avant entraînement
if torch.cuda.is_available():
    print(f"Mémoire GPU avant entraînement: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

# Fonction de monitoring GPU pour TrackIO
def log_gpu_stats():
    if torch.cuda.is_available():
        gpu_stats = {
            "gpu_memory_allocated_gb": torch.cuda.memory_allocated() / 1e9,
            "gpu_memory_reserved_gb": torch.cuda.memory_reserved() / 1e9,
            "gpu_utilization_percent": torch.cuda.utilization(),
        }
        try:
            # Tentative de lecture de la température et de la puissance
            gpu_stats.update({
                "gpu_temperature_c": torch.cuda.temperature(),
                "gpu_power_watts": torch.cuda.power_draw(),
            })
        except:
            pass
        trackio.log(gpu_stats)

# Log initial
log_gpu_stats()
trackio.log({
    "training_start_time": time.time(),
    "dataset_size": len(dataset_dict['train']),
    "validation_size": len(dataset_dict['validation'])
})

# Lancement de l'entraînement
trainer_stats = trainer.train()

print("\\n✅ Entraînement terminé!")
print(f"   - Perte finale: {trainer_stats.training_loss:.4f}")
print(f"   - Temps d'entraînement: {trainer_stats.metrics.get('train_runtime', 0):.1f} secondes")

# Suivi de la mémoire après entraînement
if torch.cuda.is_available():
    print(f"Mémoire GPU après entraînement: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

# Log des résultats finaux
log_gpu_stats()
trackio.log({
    "training_end_time": time.time(),
    "final_training_loss": trainer_stats.training_loss,
    "training_runtime_seconds": trainer_stats.metrics.get('train_runtime', 0),
    "training_samples_per_second": trainer_stats.metrics.get('train_samples_per_second', 0),
    "total_steps": trainer.state.global_step
})

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


🚀 Démarrage de l'entraînement LoRA...
   - Dataset: 32 exemples
   - Paramètres LoRA: rank=16, alpha=16
   - Learning rate: 0.0002
   - Epochs: 3
   - TrackIO: Activé\n
Mémoire GPU avant entraînement: 3.7 GB


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32 | Num Epochs = 3 | Total steps = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
1,1.275000,0
2,1.510600,No Log
3,1.292400,No Log
4,1.350100,No Log
5,1.352300,No Log
6,1.031100,No Log
7,1.001400,No Log
8,0.956400,No Log
9,1.062900,No Log
10,0.884900,No Log


* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
\n✅ Entraînement terminé!
   - Perte finale: 1.1228
   - Temps d'entraînement: 38.5 secondes
Mémoire GPU après entraînement: 3.8 GB


## 8. Évaluation du modèle

Nous évaluons les performances du modèle fine-tuné sur le jeu de validation.

In [10]:
# Évaluation sur le jeu de validation avec TrackIO
print("📊 Évaluation du modèle sur le jeu de validation...")
eval_results = trainer.evaluate()

print("\\nRésultats de l'évaluation:")
for key, value in eval_results.items():
    if isinstance(value, (int, float)):
        print(f"   - {key}: {value:.4f}")

# Log des résultats d'évaluation
trackio.log({
    "eval_loss": eval_results.get("eval_loss", 0),
    "eval_runtime": eval_results.get("eval_runtime", 0),
    "eval_samples_per_second": eval_results.get("eval_samples_per_second", 0),
    "eval_steps_per_second": eval_results.get("eval_steps_per_second", 0)
})

# Visualisation de la courbe d'apprentissage
if hasattr(trainer, "state") and trainer.state.log_history:
    log_history = trainer.state.log_history
    losses = [log.get('loss', 0) for log in log_history if 'loss' in log]
    steps = [log.get('step', 0) for log in log_history if 'loss' in log]
    
    plt.figure(figsize=(10, 6))
    plt.plot(steps, losses, 'b-', label='Training Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.title('Courbe d\\apprentissage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Log des métriques de la courbe d'apprentissage
    if losses:
        trackio.log({
            "training_loss_min": min(losses),
            "training_loss_final": losses[-1],
            "training_loss_improvement": losses[0] - losses[-1] if len(losses) > 1 else 0
        })

Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


📊 Évaluation du modèle sur le jeu de validation...


\nRésultats de l'évaluation:
   - eval_loss: 1.1588
   - eval_runtime: 0.4112
   - eval_samples_per_second: 9.7280
   - eval_steps_per_second: 4.8640
   - epoch: 3.0000


<Figure size 1000x600 with 1 Axes>

## 9. Test d'inférence

Nous testons le modèle fine-tuné sur quelques exemples pour évaluer sa capacité à générer des réponses en français.

In [11]:
# Configuration du modèle pour l'inférence optimisée avec Unsloth
FastLanguageModel.for_inference(model)

# Fonction de génération optimisée avec Unsloth
def generate_response(instruction: str, max_new_tokens: int = 512) -> str:
    """
    Génère une réponse à partir d'une instruction avec paramètres optimisés Unsloth.
    """
    # Formatage du prompt selon le template utilisé pendant l'entraînement
    prompt = alpaca_prompt.format(
        instruction,
        "",  # Input vide
        ""  # Laisser vide pour la génération
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Génération avec paramètres optimisés pour le français
    # Utilisation des optimisations Unsloth
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True,  # Activé par défaut avec Unsloth
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # Extraire seulement la réponse générée
    response = response.split("### Response:")[1].strip()
    return response

# Test avec exemples variés en français
test_instructions = [
    "Qu'est-ce que l'intelligence artificielle?",
    "Explique le principe du fine-tuning avec LoRA.",
    "Quels sont les avantages de QLoRA par rapport au fine-tuning classique?",
    "Comment fonctionne l'apprentissage par renforcement?",
    "Qu'est-ce que la quantification 4-bit dans les LLMs?"
]

print("🧪 Tests d'inférence:\n")

# Log des tests d'inférence
inference_results = []

for i, instruction in enumerate(test_instructions):
    print(f"--- Test {i+1} ---")
    print(f"Instruction: {instruction}")
    
    start_time = time.time()
    response = generate_response(instruction)
    inference_time = time.time() - start_time
    
    print(f"Réponse: {response[:300]}...")
    print(f"Temps de génération: {inference_time:.2f} secondes\n")
    
    # Stocker les résultats pour le logging
    inference_results.append({
        "test_id": i + 1,
        "instruction": instruction,
        "response_length": len(response),
        "inference_time": inference_time,
        "tokens_per_second": len(response.split()) / inference_time if inference_time > 0 else 0
    })

# Log des résultats d'inférence
trackio.log({
    "inference_tests": inference_results,
    "avg_inference_time": sum(r["inference_time"] for r in inference_results) / len(inference_results),
    "avg_tokens_per_second": sum(r["tokens_per_second"] for r in inference_results) / len(inference_results)
})

# Afficher les statistiques d'inférence
print("\n📊 Statistiques d'inférence:")
print(f"   - Temps moyen: {sum(r['inference_time'] for r in inference_results) / len(inference_results):.2f}s")
print(f"   - Tokens/sec moyen: {sum(r['tokens_per_second'] for r in inference_results) / len(inference_results):.1f}")

🧪 Tests d'inférence:

--- Test 1 ---
Instruction: Qu'est-ce que l'intelligence artificielle?


AttributeError: 'NoneType' object has no attribute 'shape'

## 10. Analyse des compromis LoRA

Nous analysons les avantages et inconvénients de l'approche LoRA.

In [12]:
# Analyse des paramètres entraînables
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
percentage = (trainable_params / total_params) * 100

print("📊 Analyse des paramètres:")
print(f"   - Paramètres totaux: {total_params:,}")
print(f"   - Paramètres entraînables: {trainable_params:,}")
print(f"   - Pourcentage entraînable: {percentage:.2f}%")

# Comparaison des approches
comparison_data = {
    "Approche": ["Full Fine-tuning", "LoRA (r=16)", "LoRA (r=8)", "LoRA (r=32)"],
    "Paramètres entraînables": [
        f"{total_params:,}",
        f"{trainable_params:,}",
        f"{trainable_params//2:,}",
        f"{trainable_params*2:,}"
    ],
    "Mémoire GPU estimée": ["~24GB", "~8GB", "~6GB", "~10GB"],
    "Temps d'entraînement": ["100%", "~30%", "~20%", "~40%"],
    "Performance relative": ["100%", "~95%", "~85%", "~98%"]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n🔄 Comparaison des approches:")
print(comparison_df.to_string(index=False))

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des types de paramètres
param_types = ['Entraînables', 'Gelés']
param_counts = [trainable_params, total_params - trainable_params]

axes[0].pie(param_counts, labels=param_types, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribution des paramètres')

# Efficacité mémoire
approaches = ['Full FT', 'LoRA r=16', 'LoRA r=8', 'LoRA r=32']
memory_usage = [100, 33, 25, 42]  # Pourcentage du full fine-tuning

axes[1].bar(approaches, memory_usage, color=['red', 'green', 'blue', 'orange'])
axes[1].set_ylabel('Mémoire GPU relative (%)')
axes[1].set_title('Efficacité mémoire')
axes[1].set_ylim(0, 120)

# Ajout des valeurs sur les barres
for i, v in enumerate(memory_usage):
    axes[1].text(i, v + 2, f'{v}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

📊 Analyse des paramètres:
   - Paramètres totaux: 2,539,650,560
   - Paramètres entraînables: 33,030,144
   - Pourcentage entraînable: 1.30%

🔄 Comparaison des approches:
        Approche Paramètres entraînables Mémoire GPU estimée Temps d'entraînement Performance relative
Full Fine-tuning           2,539,650,560               ~24GB                 100%                 100%
     LoRA (r=16)              33,030,144                ~8GB                 ~30%                 ~95%
      LoRA (r=8)              16,515,072                ~6GB                 ~20%                 ~85%
     LoRA (r=32)              66,060,288               ~10GB                 ~40%                 ~98%


<Figure size 1400x500 with 2 Axes>

## 11. Sauvegarde du modèle

Nous sauvegardons le modèle fine-tuné pour une utilisation future.

In [13]:
# Création du dossier de sauvegarde
model_save_path = Path("../models/lora_qwen3_french")
model_save_path.mkdir(parents=True, exist_ok=True)

print(f"💾 Sauvegarde du modèle dans {model_save_path}...")

# Sauvegarde du modèle LoRA avec Unsloth
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print("✅ Adaptateurs LoRA sauvegardés avec succès!")

# Sauvegarde de la configuration
config = {
    "base_model": model_name,
    "lora_config": lora_config,
    "training_args": {
        "num_train_epochs": training_args.num_train_epochs,
        "learning_rate": training_args.learning_rate,
        "per_device_train_batch_size": training_args.per_device_train_batch_size,
    },
    "dataset_info": {
        "train_size": len(dataset_dict['train']),
        "val_size": len(dataset_dict['validation']),
        "source": "synthetic_dataset"
    },
    "results": {
        "final_loss": trainer_stats.training_loss,
        "eval_results": eval_results
    }
}

with open(model_save_path / "config.json", 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

# Sauvegarde en 16-bit pour l'inférence plus rapide
print("\n💾 Sauvegarde en 16-bit pour l'inférence...")
model.save_pretrained_merged(
    model_save_path / "merged_16bit",
    tokenizer,
    save_method="merged_16bit"
)
print("✅ Sauvegarde 16-bit terminée!")

# Sauvegarde en 4-bit pour la taille minimale
print("\n💾 Sauvegarde en 4-bit pour la taille minimale...")
model.save_pretrained_merged(
    model_save_path / "merged_4bit",
    tokenizer,
    save_method="merged_4bit_forced"
)
print("✅ Sauvegarde 4-bit terminée!")

# Affichage des tailles des modèles
print("\n📊 Tailles des modèles sauvegardés:")
for folder in ["", "merged_16bit", "merged_4bit"]:
    folder_path = model_save_path / folder if folder else model_save_path
    if folder_path.exists():
        size = sum(os.path.getsize(folder_path / f) for f in os.listdir(folder_path) 
                  if os.path.isfile(folder_path / f)) / 1e6
        print(f"   - {folder if folder else 'LoRA adapters'}: {size:.1f} MB")

# Log des informations de sauvegarde
trackio.log({
    "model_saved": True,
    "save_path": str(model_save_path),
    "saved_formats": ["lora_adapters", "merged_16bit", "merged_4bit"],
    "final_training_loss": trainer_stats.training_loss
})

💾 Sauvegarde du modèle dans ../models/lora_qwen3_french...
✅ Adaptateurs LoRA sauvegardés avec succès!

💾 Sauvegarde en 16-bit pour l'inférence...


config.json: 0.00B [00:00, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████████████████████████████████████████████████████████████                                                             | 1/2 [00:30<00:30, 30.27s/it]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:45<00:00, 22.89s/it]


✅ Sauvegarde 16-bit terminée!

💾 Sauvegarde en 4-bit pour la taille minimale...
Unsloth: Merging LoRA weights into 4bit model...
Unsloth: Merging finished.
Unsloth: Found skipped modules: ['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.3.self_attn.q_proj', 'model.layers.3.self_attn.k_proj', 'model.layers.3.self_attn.v_proj', 'model.layers.3.self_attn.o_proj', 'model.layers.3.mlp.gate_proj', 'model.layers.3.mlp.up_proj', 'model.layers.3.mlp.down_proj', 'model.layers.5.mlp.gate_proj', 'model.layers.5.mlp.up_proj', 'model.layers.5.mlp.down_proj', 'model.layers.6.self_attn.q_proj', 'model.layers.6.self_attn.k_proj', 'model.layers.6.self_attn.v_proj', 'model.layers.6.self_attn.o_proj', 'model.layers.6

## 12. Préparation pour la publication sur Hugging Face

Nous préparons le modèle pour sa publication sur Hugging Face Hub.

In [14]:
# Création du README pour Hugging Face
readme_content = f"""---
library_name: transformers
license: mit
base_model: {model_name}
tags:
- trl
- sft
- generated_from_trainer
- french
- lora
- qwen3
model-index:
- name: Qwen3-4B-French-LoRA
  results: []
---

# Qwen3-4B-French-LoRA

## Description

Ce modèle est une version fine-tunée de Qwen3-4B-Instruct spécialisée pour le français. Il a été entraîné sur un dataset synthétique d'instructions-réponses en français.

## Détails d'entraînement

- **Base Model**: {model_name}
- **Training Framework**: Unsloth + TRL
- **Quantization**: 4-bit (QLoRA)
- **LoRA Rank**: {lora_config['r']}
- **LoRA Alpha**: {lora_config['lora_alpha']}
- **Training Epochs**: {training_args.num_train_epochs}
- **Learning Rate**: {training_args.learning_rate}
- **Dataset Size**: {len(dataset_dict['train'])} examples

## Performance

- **Training Loss**: {trainer_stats.training_loss:.4f}
- **Validation Loss**: {eval_results.get('eval_loss', 'N/A')}

## Utilisation

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Chargement du modèle
model_path = "votre-username/Qwen3-4B-French-LoRA"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Formatage du prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{{}}

### Input:
{{}}

### Response:
{{}}"""

# Génération
prompt = alpaca_prompt.format(
    "Qu'est-ce que l'intelligence artificielle?",
    "",
    ""
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
```

## Limitations

- Le modèle est spécialisé pour le français
- Performances limitées pour les tâches nécessitant un raisonnement complexe
- Peut générer des informations incorrectes (hallucinations)

## Éthique

Ce modèle est destiné à des fins éducatives et de recherche. Les utilisateurs doivent être conscients des biais potentiels et utiliser le modèle de manière responsable.
"""

# Sauvegarde du README
with open(model_save_path / "README.md", 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("📝 README créé pour Hugging Face Hub")

# Instructions pour la publication
print("\n🚀 Instructions pour la publication sur Hugging Face Hub:")
print("1. Créez un compte sur Hugging Face: https://huggingface.co/join")
print("2. Installez l'CLI Hugging Face: pip install 'huggingface_hub[cli]'")
print("3. Connectez-vous: huggingface-cli login")
print("4. Créez un nouveau repository: huggingface-cli repo create Qwen3-4B-French-LoRA --type model")
print("5. Uploadez votre modèle: huggingface-cli upload ./models/lora_qwen3_french votre-username/Qwen3-4B-French-LoRA")

# Script de publication automatisé
publish_script = f"""#!/bin/bash
# Script pour publier le modèle sur Hugging Face Hub

# Remplacez VOTRE_USERNAME par votre username Hugging Face
USERNAME="VOTRE_USERNAME"
MODEL_NAME="Qwen3-4B-French-LoRA"
MODEL_PATH="./models/lora_qwen3_french"

echo "Publication du modèle $MODEL_NAME sur Hugging Face Hub..."

# Vérification que le modèle existe
if [ ! -d "$MODEL_PATH" ]; then
    echo "Erreur: Le modèle n'existe pas dans $MODEL_PATH"
    exit 1
fi

# Upload du modèle
huggingface-cli upload $MODEL_PATH $USERNAME/$MODEL_NAME \
    --repo-type model \
    --private  # Changez à --public pour un modèle public

echo "✅ Modèle publié avec succès!"
echo "🔗 Visitez: https://huggingface.co/$USERNAME/$MODEL_NAME"
"""

with open(model_save_path / "publish.sh", 'w') as f:
    f.write(publish_script)

# Rendre le script exécutable
os.chmod(model_save_path / "publish.sh", 0o755)
print("\n📜 Script de publication créé: publish.sh")

SyntaxError: invalid syntax (2040911683.py, line 52)

## 13. Résumé et conclusions

In [15]:
# Résumé final et completion de TrackIO
print("🎉 TP de fine-tuning LoRA avec Unsloth terminé!\n")

print("📊 Résumé des résultats:")
print(f"   - Modèle de base: {model_name}")
print(f"   - Paramètres LoRA: rank={lora_config['r']}, alpha={lora_config['lora_alpha']}")
print(f"   - Dataset d'entraînement: {len(dataset_dict['train'])} exemples")
print(f"   - Époques d'entraînement: {training_args.num_train_epochs}")
print(f"   - Perte finale: {trainer_stats.training_loss:.4f}")
print(f"   - Perte validation: {eval_results.get('eval_loss', 'N/A')}")
print(f"   - Mémoire GPU utilisée: {torch.cuda.memory_allocated() / 1e9:.1f} GB")
print(f"   - TrackIO: Activé et loggé")

print("\n📁 Fichiers créés:")
print(f"   - {model_save_path.name}/ (adaptateurs LoRA)")
print(f"   - {model_save_path.name}/merged_16bit/ (modèle fusionné 16-bit)")
print(f"   - {model_save_path.name}/merged_4bit/ (modèle fusionné 4-bit)")
print(f"   - {model_save_path.name}/README.md (documentation)")
print(f"   - {model_save_path.name}/publish.sh (script de publication)")

print("\n🔍 Points clés appris:")
print("   1. LoRA permet de fine-tuner avec ~1% des paramètres")
print("   2. QLoRA réduit significativement la mémoire GPU")
print("   3. Unsloth accélère l'entraînement de 2-3x")
print("   4. TrackIO permet un suivi précis des expériences")
print("   5. Plusieurs formats de sauvegarde pour différents cas d'usage")

print("\n🚀 Prochaines étapes:")
print("   1. Publier le modèle sur Hugging Face Hub")
print("   2. Tester avec des données réelles")
print("   3. Expérimenter avec différents paramètres LoRA")
print("   4. Essayer d'autres techniques de fine-tuning")
print("   5. Analyser les résultats dans TrackIO dashboard")

# Finalisation de TrackIO
trackio.log({
    "experiment_completed": True,
    "completion_time": time.time(),
    "model_saved": True,
    "total_experiment_time": time.time() - trackio._start_time if hasattr(trackio, '_start_time') else 0,
    "key_learnings": [
        "Unsloth provides significant speedup for LoRA training",
        "QLoRA enables training on consumer GPUs",
        "Multiple save formats provide flexibility for deployment",
        "TrackIO enables experiment tracking and comparison"
    ]
})

# Completion de l'expérience
trackio.finish()

print("\n📊 TrackIO: Expérience complétée et sauvegardée!")
print("   - Visualisez les résultats avec: trackio show")
print("   - Ou consultez le dashboard TrackIO")

🎉 TP de fine-tuning LoRA avec Unsloth terminé!

📊 Résumé des résultats:
   - Modèle de base: Qwen/Qwen3-4B-Instruct-2507
   - Paramètres LoRA: rank=16, alpha=16
   - Dataset d'entraînement: 32 exemples
   - Époques d'entraînement: 3
   - Perte finale: 1.1228
   - Perte validation: 1.1588257551193237
   - Mémoire GPU utilisée: 3.8 GB
   - TrackIO: Activé et loggé

📁 Fichiers créés:
   - lora_qwen3_french/ (adaptateurs LoRA)
   - lora_qwen3_french/merged_16bit/ (modèle fusionné 16-bit)
   - lora_qwen3_french/merged_4bit/ (modèle fusionné 4-bit)
   - lora_qwen3_french/README.md (documentation)
   - lora_qwen3_french/publish.sh (script de publication)

🔍 Points clés appris:
   1. LoRA permet de fine-tuner avec ~1% des paramètres
   2. QLoRA réduit significativement la mémoire GPU
   3. Unsloth accélère l'entraînement de 2-3x
   4. TrackIO permet un suivi précis des expériences
   5. Plusieurs formats de sauvegarde pour différents cas d'usage

🚀 Prochaines étapes:
   1. Publier le modèle su

## 15. Pour aller plus loin

### Expérimentations suggérées

1. **Variation du rank LoRA**: Tester r=8, r=16, r=32 et comparer les performances
2. **Différents taux d'apprentissage**: 1e-4, 2e-4, 5e-4
3. **Augmentation du dataset**: Générer plus de données synthétiques

### Optimisations possibles

1. **Batch processing**: Augmenter la taille du batch si mémoire permet
2. **Gradient checkpointing**: Activé par défaut dans Unsloth
3. **Mixed precision**: Utiliser bf16 si disponible
4. **Data parallelism**: Pour multi-GPU

### Évaluation approfondie

1. **Benchmarks automatiques**: Utiliser des benchmarks français
2. **Évaluation humaine**: Qualité des réponses générées
3. **Comparaison baseline**: Contre le modèle de base
4. **Analyse d'erreurs**: Comprendre les faiblesses du modèle

### Déploiement

2. **Optimisation inférence**: Regarder vLLM et SGLang
3. **Quantification post-entraînement**: GGUF, AWQ

### TrackIO avancé

Pour analyser vos expériences plus en détail:

```python
# Comparer plusieurs expériences
trackio.compare_experiments([
    "qwen3-4b-lora-french-1",
    "qwen3-4b-lora-french-2",
    "qwen3-4b-lora-french-3"
])

# Exporter les résultats
trackio.export_results("experiment_results.csv")

# Visualiser les courbes d'apprentissage
trackio.plot_training_curves()
```